# Populate the portal using the API
Import the data prepared in `filter_papers_by_software.ipynb` into the data structure setup in `import_wikidata_properties.ipynb`.

The API documentation is [here](https://www.wikidata.org/w/api.php?action=help&modules=wbeditentity)

## Login to the Wikibase

### Network settings
* Make sure the wikibase is running, e.g. using [MaRDI4NFDI/portal-compose](https://github.com/MaRDI4NFDI/portal-compose)
* Make sure this jupyter notebook is in the same network as the wiki. This is done in docker-compose

```
networks:
  default:
    external: true
    name: portal-compose_default
```

Networks can be listed using `docker network ls`. Here, "portal-compose_default" is the name of the network started by portal-compose.
* Verify that this notebook is in the correct network `docker network inspect portal-compose_default`

The wiki is then accessible from the notebook container at `http://mardi-wikibase`.

### Credentials
* Login to the wiki as admin
* Go to Special:BotPasswords, create a bot user, call it "import", grant it "High-volume editing", "Edit existing pages", "Create, edit, and move pages"
* Copy `data/credentials.tpl` to `data/credentials.ini`. Replace the username and password by those of the newly created bot user (make sure not to commit this file)

In [39]:
import requests
import json 
import configparser

# url of the API endpoint
WIKIBASE_API = 'http://mardi-wikibase/w/api.php?format=json'

# bot username and password
config = configparser.ConfigParser()
config.sections()
config.read('data/credentials.ini')
username = config['default']['username']
password = config['default']['password']

# Cookie available after Login
cookies = None

print(username, password)
def login():
    # get a login token
    TOKEN_ENDPOINT = '{}&action=query&meta=tokens&type=login'.format(WIKIBASE_API)
    resp = requests.get(TOKEN_ENDPOINT)
    logintoken = json.loads(resp.text)['query']['tokens']['logintoken']
    cookies = resp.cookies

    # use with bot user
    LOGIN_ENDPOINT = '{}&action=login'.format(WIKIBASE_API)
    resp = requests.post(
        LOGIN_ENDPOINT, 
        data={'lgname':username, 'lgpassword': password, 'lgtoken': logintoken, 'format': 'json'},
    cookies = cookies)
    print(resp.text)

login()

Admin@import 0gs16d2gh9cj2gckujo9ui8e04jpqps2
{"login":{"result":"Success","lguserid":1,"lgusername":"Admin"}}


## Import the authors list
A subsample of the authors list was created in notebook `filter_papers_by_software.ipyb`. This list contains the authors of a papers related to the first 1000 software entries in the list of softwares (`data/swMath-software-list.csv`). The list of authors is in file `data/all_authors.csv.zip`. 

In [1]:
# load the list of authors
import pandas as pd

# load the list of zbMath authors
authors_df = pd.read_csv('data/all_authors.csv.zip')
#authors_df.head()

Map the columns to MaRDI-Portal properties, reformat the data according to [the CSV file syntax expected by Quickstatements](https://www.wikidata.org/wiki/Help:QuickStatements#CSV_file_syntax). 

In [2]:
from datetime import datetime

import_authors_df = pd.DataFrame()
import_authors_df['qid'] = len(authors_df) * [''] # leave empty to create new item
import_authors_df['Len'] = authors_df['author_name']
import_authors_df['P31'] = len(authors_df) * ['Q5'] # instance of 'human'
import_authors_df['P1556'] = authors_df['author_id'] # zbMath author id
import_authors_df['#'] = len(authors_df) * ['{}: imported from zbMath Open API'.format(datetime.now())]

In [3]:
import_authors_df.head()

,qid,Len,P31,P1556,#
0,,"Aardal, Karen",Q5,aardal.karen-i,2022-01-28 16:50:49.379647: imported from zbMa...
1,,"Aarts, Gert",Q5,aarts.gert,2022-01-28 16:50:49.379647: imported from zbMa...
2,,"Abad, Alberto",Q5,abad.alberto-j,2022-01-28 16:50:49.379647: imported from zbMa...
3,,"Abada, Asmaa",Q5,abada.asmaa,2022-01-28 16:50:49.379647: imported from zbMa...
4,,"Abánades, Miguel A.",Q5,abanades.miguel-angel,2022-01-28 16:50:49.379647: imported from zbMa...
